## 1. 데이터 불러오기

### 필수 라이브러리

In [4]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)

ROOT_DIR = "../data"
RANDOM_STATE = 110

# Load data
train_data = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))


In [5]:
# Under-sampling
normal_ratio = 1.0  # 1.0 means 1:1 ratio

df_normal = train_data[train_data["target"] == "Normal"]
df_abnormal = train_data[train_data["target"] == "AbNormal"]

num_normal = len(df_normal)
num_abnormal = len(df_abnormal)
print(f"  Total: Normal: {num_normal}, AbNormal: {num_abnormal}")


  Total: Normal: 38156, AbNormal: 2350


In [6]:
df_normal = df_normal.sample(n=int(num_abnormal * normal_ratio), replace=False, random_state=RANDOM_STATE)
df_concat = pd.concat([df_normal, df_abnormal], axis=0).reset_index(drop=True)
print(df_concat.value_counts("target"))

# Split data
df_train, df_val = train_test_split(
    df_concat,
    test_size=0.3,
    stratify=df_concat["target"],
    random_state=RANDOM_STATE,
)

def print_stats(df: pd.DataFrame):
    num_normal = len(df[df["target"] == "Normal"])
    num_abnormal = len(df[df["target"] == "AbNormal"])
    print(f"  Total: Normal: {num_normal}, AbNormal: {num_abnormal}" + f" ratio: {num_abnormal/num_normal}")

# Print statistics
print(f"  \tAbnormal\tNormal")
print_stats(df_train)
print_stats(df_val)

target
AbNormal    2350
Normal      2350
Name: count, dtype: int64
  	Abnormal	Normal
  Total: Normal: 1645, AbNormal: 1645 ratio: 1.0
  Total: Normal: 705, AbNormal: 705 ratio: 1.0


In [7]:
# Model definition
model = CatBoostClassifier(random_state=RANDOM_STATE, iterations=1000, depth=6, learning_rate=0.1, loss_function='Logloss', verbose=200)

# Prepare training data
features = []

for col in df_train.columns:
    try:
        df_train[col] = df_train[col].astype(int)
        features.append(col)
    except:
        continue

train_x = df_train[features]
train_y = df_train["target"]

# Train model
model.fit(train_x, train_y)

0:	learn: 0.6871760	total: 60.1ms	remaining: 1m
200:	learn: 0.5076422	total: 472ms	remaining: 1.88s
400:	learn: 0.3992621	total: 877ms	remaining: 1.31s
600:	learn: 0.3308934	total: 1.29s	remaining: 857ms
800:	learn: 0.2803897	total: 1.7s	remaining: 422ms
999:	learn: 0.2410478	total: 2.1s	remaining: 0us


In [8]:
# Load test data
test_data = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"))

# Prepare test data
df_test_x = test_data[features]

for col in df_test_x.columns:
    try:
        df_test_x[col] = df_test_x[col].astype(int)
    except:
        continue

# Predict
test_pred = model.predict(df_test_x)

# Prepare submission
df_sub = pd.read_csv("../submission.csv")
df_sub["target"] = test_pred
df_sub.to_csv("submission.csv", index=False)

/var/folders/pg/xg71_x050fx3k72nr___j2400000gn/T/ipykernel_82864/1620896495.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_x[col] = df_test_x[col].astype(int)
/var/folders/pg/xg71_x050fx3k72nr___j2400000gn/T/ipykernel_82864/1620896495.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_x[col] = df_test_x[col].astype(int)
/var/folders/pg/xg71_x050fx3k72nr___j2400000gn/T/ipykernel_82864/1620896495.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

FileNotFoundError: [Errno 2] No such file or directory: 'submission.csv'